### Word In Visual Genome Process

In [1]:
#!pip install --upgrade pip

In [2]:
#!pip install --upgrade regex

In [3]:
#!pip install --upgrade pandas

In [4]:
#!pip install --upgrade yellowbrick

In [5]:
#!pip install --upgrade scipy

In [6]:
#import re
#print(re.__version__)

In [7]:
#import pandas
#print(pandas.__version__)

In [8]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [9]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [10]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-1-Word In Visual Genome"

Path(path).mkdir(parents=True, exist_ok=True)

In [11]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [12]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

#### Visual Genome Data

In [13]:
df_genome_objects = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Objects_Analysis.csv")
df_genome_objects.drop("image_url", inplace=True, axis=1)
df_genome_objects

,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,1058549,trees,1,tree.n.01,NaN,557,799,0,0
1,1058534,sidewalk,1,sidewalk.n.01,5046,290,722,78,308
2,1058508,building,1,building.n.01,NaN,538,222,1,0
3,1058539,street,1,street.n.01,3798578,258,359,439,283
4,1058543,wall,1,wall.n.01,NaN,535,135,0,1
...,...,...,...,...,...,...,...,...,...
2516934,3506171,number,2417997,numeral.n.01,NaN,24,28,188,228
2516935,3786799,plate,2417997,plate.n.04,NaN,54,89,238,233
2516936,3103226,road,2417997,road.n.01,NaN,83,90,312,247
2516937,3094589,road,2417997,road.n.01,NaN,121,495,2,208


In [14]:
df_genome_objects["names"].nunique()

82825

In [15]:
df_genome_attributes = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Attributes_Analysis.csv")
df_genome_attributes

,object_id,names,image_id,attributes,synsets,height,width,x_koor,y_koor
0,1058498,clock,1,"green , tall",clock.n.01,339,79,421,91
1,5046,street,1,sidewalk,street.n.01,262,714,77,328
2,5045,shade,1,NaN,shade.n.01,192,274,119,338
3,1058529,man,1,NaN,man.n.01,262,60,238,249
4,5048,sneakers,1,grey,gym_shoe.n.01,26,52,243,489
...,...,...,...,...,...,...,...,...,...
3802369,3422177,sky,2417997,"crystal clear , blue",sky.n.01,170,497,1,2
3802370,3103226,road,2417997,NaN,road.n.01,83,90,312,247
3802371,2712437,doors,2417997,NaN,door.n.01,144,50,98,127
3802372,3140197,bus,2417997,green,bus.n.01,155,225,29,109


In [16]:
df_genome_relationships = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Result/3-Visual Genome Process/Visual_Genome_Relationships_Analysis.csv")
df_genome_relationships

,image_id,obj_names,obj_object_id,obj_synsets,obj_merged_object_ids,obj_height,obj_width,obj_x_koor,obj_y_koor,relationships_id,synsets,sub_name,sub_object_id,sub_synsets,sub_height,sub_width,sub_x_koor,sub_y_koor,predicate
0,1,sidewalk,1058534,sidewalk.n.01,5046,290,722,78,308,15927,along.r.01,shade,5045,shade.n.01,192,274,119,338,on
1,1,shoes,1058525,shoe.n.01,5048,28,48,388,485,15928,wear.v.01,man,1058529,man.n.01,262,60,238,249,wears
2,1,NaN,5050,headlight.n.01,NaN,15,23,514,366,15929,have.v.01,car,5049,car.n.01,98,74,479,315,has
3,1,NaN,1058508,building.n.01,NaN,536,218,1,2,15930,along.r.01,sign,1058507,sign.n.02,182,88,118,13,on
4,1,NaN,1058534,sidewalk.n.01,NaN,266,722,77,331,15931,along.r.01,tree trunk,5055,trunk.n.01,327,87,622,234,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2316099,2417997,NaN,3103225,bus.n.01,NaN,193,289,14,110,4245033,along.r.01,plate,3786799,plate.n.04,54,89,238,233,on
2316100,2417997,NaN,3786800,desert.n.01,NaN,119,157,301,211,4245034,in.r.01,NaN,3103225,bus.n.01,182,287,14,110,in
2316101,2417997,NaN,3103226,road.n.01,NaN,83,90,312,247,3722727,along.r.01,bus,3103225,bus.n.01,193,289,14,110,on
2316102,2417997,NaN,2712437,door.n.01,NaN,144,50,98,127,3880229,NaN,bus,3103225,bus.n.01,193,289,14,110,with


#### Word Search In Genome Data

In [17]:
#df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/2-Word Select Process/{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
#df_word_select

In [18]:
#word_list = ['an','anlama','doğum','gidiyor','günün','istediğim','istediğin','istediğiniz','kutlu','olmuş','soru','söylemek','zamanki','zor','şuna']

In [19]:
#df_word_raw = df_word_select[df_word_select["word"].isin(word_list)]
#df_word_raw.reset_index(drop=True, inplace=True)
#df_word_raw.drop(["frequency","stem"], inplace=True, axis=1)
#df_word_raw

In [20]:
df_word_raw = pd.read_excel("Turkish_45000_Process.xlsx")
df_word_raw

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
0,NUM,NaN,bir,bir,bir,a,a,18835735
1,PRON,NaN,bu,bu,bu,this,this,11062659
2,PRON,Q,ne,ne,ne,what,what,8025880
3,CCONJ,NaN,ve,ve,ve,and,and,7766036
4,ADP,NaN,için,için,için,for,for,5484109
...,...,...,...,...,...,...,...,...
995,ADJ,NaN,resmi,resmi,resmi,formal,formal,68287
996,VERB,NaN,veriyor,ver,ver,giving,give,68163
997,NOUN,NaN,okul,okul,oku,school,school,68160
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [21]:
df_word_raw_word_trans_list = list(set(df_word_raw["word_en_translate"]))

In [22]:
len(df_word_raw_word_trans_list)

818

In [23]:
##df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 20)
#df_object_search_word_result = word_group_dataframe_all(df_genome_objects, df_word_raw_word_trans_list, "names")
#df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
#df_object_search_word_result

In [24]:
df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 50)
df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_object_search_word_result

,word_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,beautiful,4365802,beautiful,4225,NaN,NaN,173,304,97,396
1,beautiful,3882633,beautiful,2365754,NaN,NaN,139,220,252,128
2,beautiful,3866374,beautiful,2368895,NaN,NaN,31,61,189,78
3,beautiful,1363857,beautiful,2378847,NaN,NaN,47,29,240,246
4,beautiful,399088,beautiful,2401462,NaN,NaN,27,45,394,132
...,...,...,...,...,...,...,...,...,...,...
2913,next,360511,next,2411108,NaN,NaN,21,75,140,289
2914,next,573077,next,2376410,NaN,NaN,70,123,182,367
2915,next,457432,next,2395154,NaN,NaN,118,92,291,101
2916,next,507444,next,2388987,NaN,NaN,97,180,0,206


In [ ]:
df_object_search_word_result.to_excel("", index=False)

In [25]:
df_object_search_word_result["word_en_translate"].nunique()

389

In [26]:
word_find_list = list(set(df_object_search_word_result["word_en_translate"]))

In [27]:
df_word_raw_for_lemma = df_word_raw[~df_word_raw["word_en_translate"].isin(word_find_list)]
df_word_raw_for_lemma

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
5,AUX,NaN,mi,mi,mi,is it,is it,5362714
11,VERB,NaN,var,var,var,there is/are,there is/are,4389551
13,AUX,NaN,mı,mı,mı,is it,is it,4001316
20,ADP,NaN,kadar,kadar,kadar,until,until,2697900
21,PRON,NaN,bana,ben,ben,to me,i,2659182
...,...,...,...,...,...,...,...,...
987,NOUN,NaN,başıma,baş,baş,to me,top,68674
991,VERB,NaN,bul,bul,bul,find,find,68524
992,VERB,NaN,edeyim,et,et,let me,do,68506
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [28]:
df_word_raw_lemma_trans_list = list(set(df_word_raw_for_lemma["lemma_en_translate"]))

In [29]:
len(df_word_raw_lemma_trans_list)

232

In [30]:
df_object_search_lemma_result = word_group_dataframe(df_genome_objects, df_word_raw_lemma_trans_list, "names", 10)
df_object_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_object_search_lemma_result

,lemma_en_translate,object_id,names,image_id,synsets,merged_object_ids,height,width,x_koor,y_koor
0,this,3793044,this,525,NaN,1623894,332,300,1,266
1,this,3901012,this,2350323,NaN,"3901006 , 2118937",127,159,144,184
2,this,3901014,this,2350323,NaN,NaN,62,28,69,100
3,this,3901013,this,2350323,NaN,NaN,54,75,64,258
4,this,3901009,this,2350323,NaN,NaN,77,120,262,298
...,...,...,...,...,...,...,...,...,...,...
688,like,169634,like horses,2413623,horse.n.01,NaN,57,95,352,105
689,like,3198035,leather like,2327188,leather.n.01,NaN,31,46,158,66
690,like,496921,tree like growths,2390481,"growth.n.01 , tree.n.01",NaN,11,17,245,85
691,like,282797,steeple like tower,2407261,"steeple.n.01 , tower.n.01",NaN,46,45,148,30


In [ ]:
df_object_search_lemma_result.to_excel("", index=False)

In [31]:
df_object_search_lemma_result["lemma_en_translate"].nunique()

91

In [32]:
lemma_find_list = list(set(df_object_search_lemma_result["lemma_en_translate"]))

In [33]:
df_word_raw_remain = df_word_raw_for_lemma[~df_word_raw_for_lemma["lemma_en_translate"].isin(word_find_list)]
df_word_raw_remain

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency
5,AUX,NaN,mi,mi,mi,is it,is it,5362714
11,VERB,NaN,var,var,var,there is/are,there is/are,4389551
13,AUX,NaN,mı,mı,mı,is it,is it,4001316
20,ADP,NaN,kadar,kadar,kadar,until,until,2697900
34,NOUN,Q,neden,neden,neden,why,why,1866089
...,...,...,...,...,...,...,...,...
980,VERB,NaN,çekilin,çek,çek,withdraw,check,69201
984,VERB,NEG,istemedim,iste,iste,i didn't want,want,68907
991,VERB,NaN,bul,bul,bul,find,find,68524
998,NOUN,NaN,suçlu,suç,suç,guilty,crime,68124


In [ ]:
df_object_search_word_result_merge = pd.merge(df_object_search_word_result,df_word_raw,how="left",on="word_en_translate")
df_object_search_word_result_merge

In [ ]:
df_word_count = df_object_search_word_result_merge.groupby(["word_en_translate"])[["names"]].count().reset_index()
df_word_count.rename(columns={"names":"names_count"}, inplace=True)
df_word_count.sort_values(by="names_count",ascending=False, inplace=True)
df_word_count.reset_index(drop=True, inplace=True)
df_word_count

In [ ]:
df_object_search_word_result_merge.to_excel("1000_Words_Word_Object_Names.xlsx", index=False)

In [ ]:
#df_object_search_lemma_result = word_group_dataframe(df_genome_objects, df_word_raw_lemma_trans_list, "names", 10)
df_object_search_lemma_result = word_group_dataframe_all(df_genome_objects, df_word_raw_lemma_trans_list, "names")
df_object_search_lemma_result.rename(columns={"search_string":"lemma_en_translate"}, inplace=True)
df_object_search_lemma_result

In [ ]:
df_object_search_lemma_result["lemma_en_translate"].nunique()

In [ ]:
df_object_search_lemma_result_merge = pd.merge(df_object_search_lemma_result,df_word_raw,how="left",on="lemma_en_translate")
df_object_search_lemma_result_merge

In [ ]:
df_lemma_count = df_object_search_lemma_result_merge.groupby(["lemma_en_translate"])[["image_id"]].count().reset_index()
df_lemma_count.rename(columns={"image_id":"image_id_count"}, inplace=True)
df_lemma_count.sort_values(by="image_id_count",ascending=False, inplace=True)
df_lemma_count.reset_index(drop=True, inplace=True)
df_lemma_count

In [ ]:
df_object_search_lemma_result_merge.to_excel("1000_Words_Lemma_Object_Names.xlsx", index=False)

In [ ]:
df_lemma_count.to_excel("1000_Words_Lemma_Object_Names_Count.xlsx", index=False)

In [ ]:

lemma_count_list = list(set(df_lemma_count["lemma_en_translate"]))

In [ ]:
len(lemma_count_list)

In [ ]:
df_word_raw

In [ ]:
df_lemma_not_cover = df_word_raw[~df_word_raw["lemma_en_translate"].isin(lemma_count_list)]
df_lemma_not_cover

In [ ]:
lemma_not_cover_set = set(df_lemma_not_cover["word"])
lemma_not_cover_set

In [ ]:
#df_object_search_word_result = word_group_dataframe(df_genome_objects, df_word_raw_word_trans_list, "names", 20)
df_object_search_word_result = word_group_dataframe_all(df_genome_objects, df_word_raw_word_trans_list, "names")
df_object_search_word_result.rename(columns={"search_string":"word_en_translate"}, inplace=True)
df_object_search_word_result

In [ ]:
df_object_search_word_result["word_en_translate"].nunique()

In [ ]:
df_object_search_word_result_merge = pd.merge(df_object_search_word_result,df_word_raw,how="left",on="word_en_translate")
df_object_search_word_result_merge

In [ ]:
df_word_count = df_object_search_word_result_merge.groupby(["word_en_translate"])[["names"]].count().reset_index()
df_word_count.rename(columns={"names":"names_count"}, inplace=True)
df_word_count.sort_values(by="names_count",ascending=False, inplace=True)
df_word_count.reset_index(drop=True, inplace=True)
df_word_count

In [ ]:
df_object_search_word_result_merge.to_excel("1000_Words_Word_Object_Names.xlsx", index=False)

In [ ]:
df_word_count.to_excel("1000_Words_Word_Object_Names_Count.xlsx", index=False)

In [ ]:
word_count_set = list(set(df_word_count["word_en_translate"]))

In [ ]:
len(word_count_set)

In [ ]:
df_word_cover = df_word_raw[df_word_raw["word_en_translate"].isin(word_count_set)]
df_word_cover

In [ ]:
word_cover_set = set(df_word_cover["word"])
word_cover_set

In [ ]:
word_cover_set.intersection(lemma_not_cover_set)

In [ ]:
len(word_cover_set.intersection(lemma_not_cover_set))

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_45000_Lemma_Pos_Process.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass